In [ ]:
!pip install gluoncv mxnet-mkl>=1.4.0 --upgrade
from gluoncv import model_zoo, data, utils
'''
yolo3_darknet53
'''
# net = model_zoo.get_model('yolo3_darknet53_voc', pretrained=True)
# x, img = data.transforms.presets.yolo.load_test(im_fname, short=512)
# class_IDs, scores, bounding_boxs = net(x)
'''
ssd_512_resnet50
'''
# net = model_zoo.get_model('ssd_512_resnet50_v1_voc', pretrained=True)
# x, img = data.transforms.presets.ssd.load_test(im_fname, short=512)
# class_IDs, scores, bounding_boxes = net(x)
'''
faster_rcnn_resnet50
'''
# net = model_zoo.get_model('faster_rcnn_resnet50_v1b_voc', pretrained=True)
# x, orig_img = data.transforms.presets.rcnn.load_test(im_fname)
# box_ids, scores, bboxes = net(x)
'''
mask_rcnn_resnet50
'''
# net = model_zoo.get_model('mask_rcnn_resnet50_v1b_coco', pretrained=True)
# x, orig_img = data.transforms.presets.rcnn.load_test(im_fname)
# ids, scores, bboxes, masks = [xx[0].asnumpy() for xx in net(x)]
net = model_zoo.get_model('yolo3_darknet53_voc', pretrained=True)
print("\n       ___ pretrained object detection network is prepared___")


       ___ pretrained object detection network is prepared___


In [ ]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')

read_path = "/content/drive/My Drive/ATI/CNN_Dataset/"
save_root = "/content/drive/My Drive/ATI/"
images_path = ["train/Occupied/", "train/Empty/",  "test/Occupied/", "test/Empty/"]

for r in range(4):
    print("_____ Folder   : ", images_path[r] )
    img_path  = read_path + images_path[r]    
    for root,dirs,files in os.walk(img_path):
        i=0
        roidb = []
        save_path = save_root + str(r) + '.csv'
        print("save_path    :   ",save_path)

        for img in files:
            if img.endswith(".jpg"):
                i+=1
                if i%100==0 :
                    print("[{0}/{1}] ".format(i ,len(files)))
                
                image_path = str(img_path) + str(img)
                x, img = data.transforms.presets.yolo.load_test(image_path, short=512)
                ids, scores, bboxes = net(x)

                bbox        = [float(i) for i in bboxes[0][0].squeeze().asnumpy()]
                class_id    = [float(i) for i in ids[0][0].squeeze().asnumpy()]
                class_prob  = [float(i) for i in scores[0][0].squeeze().asnumpy()]

                temp = {}
                temp['image_path']   = image_path
                temp['y1']           = bbox[0]
                temp['x1']           = bbox[1]
                temp['y2']           = bbox[2]
                temp['x2']           = bbox[3]
                temp['class_id']     = class_id[0]
                temp['class_prob']   = class_prob[0]
                #temp['masks']       = [float(i) for i in  masks[0].squeeze().asnumpy()]
                
                if r==0 or r == 2:
                    temp['label']  = 1
                elif r == 1 or r ==3:
                    temp['label']  = 0
                
                roidb.append(temp)
        print("SAVING ... ")
        roidb = pd.DataFrame(roidb)
        roidb.to_csv( save_path , index=False)
                    
print("\n   DONE   !")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
_____ Folder   :  train/Occupied/
save_path    :    /content/drive/My Drive/ATI/0.csv
[100/1472] 
[200/1472] 
[300/1472] 
[400/1472] 
[500/1472] 
[600/1472] 
[700/1472] 
[800/1472] 
[900/1472] 
[1000/1472] 
[1100/1472] 
[1200/1472] 
[1300/1472] 
[1400/1472] 
SAVING ... 
_____ Folder   :  train/Empty/
save_path    :    /content/drive/My Drive/ATI/1.csv
[100/1306] 
[200/1306] 
[300/1306] 
[400/1306] 
[500/1306] 
[600/1306] 
[700/1306] 
[800/1306] 
[900/1306] 
[1000/1306] 
[1100/1306] 
[1200/1306] 
[1300/1306] 
SAVING ... 
_____ Folder   :  test/Occupied/
save_path    :    /content/drive/My Drive/ATI/2.csv
[100/247] 
[200/247] 
SAVING ... 
_____ Folder   :  test/Empty/
save_path    :    /content/drive/My Drive/ATI/3.csv
[100/253] 
[200/253] 
SAVING ... 

   DONE   !


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/ATI/"

train_1 = pd.read_csv(path +"0.csv")
train_2 = pd.read_csv(path +"1.csv")
test_1  = pd.read_csv(path +"2.csv")
test_2  = pd.read_csv(path +"3.csv")

train_csv = pd.concat( [ train_1 , train_2 ] )
test_csv  = pd.concat( [ test_1  , test_2 ] )

# shuffle the dataset
train = train_csv.sample(frac=1, random_state=1431)
test  = test_csv.sample(frac=1, random_state=1431)

train.to_csv(path + "train.csv", index=False )
test.to_csv(path +  "test.csv", index=False )

print("________ Train.csv SAVED________")
print("________ Test.csv  SAVED________")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
________ Train.csv SAVED________
________ Test.csv  SAVED________


In [ ]:
from glob import glob
import numpy as np
import cv2
from google.colab import drive
import pandas as pd
import pickle

drive.mount('/content/drive')

path        = "/content/drive/My Drive/DataSet/"
TRAIN_DIR   = path + "CNN_Dataset/train/"
VALID_DIR   = path + "CNN_Dataset/test/"
occupied_dir= path + "CNN_Dataset/test/Occupied/*.jpg"
empty_dir   = path + "CNN_Dataset/test/Empty/*.jpg"
trainCSV_dir= path + "CNN_Dataset/train.csv"
testCSV_dir = path + "CNN_Dataset/test.csv"

valid_images= glob(occupied_dir) + glob(empty_dir)

trainCSV = pd.read_csv(trainCSV_dir)

labels = trainCSV.label
lables_out =  open(TRAIN_DIR + "/labels.pickle","wb")
pickle.dump(labels, lables_out)
lables_out.close()

numricalData = trainCSV.iloc[:,1:7]
numricalData_out =  open(TRAIN_DIR + "/numricalData.pickle","wb")
pickle.dump(numricalData, numricalData_out)
numricalData_out.close()

imgData = []
for index, row in trainCSV.iterrows():
  #convert img to data
  mPath = row.image_path.replace("/ATI","")

  # img = image.load_img(mPath, target_size=(HEIGHT, WIDTH))
  # x = image.img_to_array(img)
  # x = np.expand_dims(x, axis=0)
  # x = preprocess_input(x)
  img = cv2.imread(mPath)
  img = cv2.resize(img,(HEIGHT, WIDTH))
  imgData.append(img)

imgData = np.array(imgData)
imgData_out =  open(TRAIN_DIR + "/imgData.pickle","wb")
pickle.dump(imgData, imgData_out)
imgData_out.close()


In [ ]:
testCSV = pd.read_csv(testCSV_dir)

labels = testCSV.label
lables_out =  open(VALID_DIR + "/labels.pickle","wb")
pickle.dump(labels, lables_out)
lables_out.close()

numricalData = testCSV.iloc[:,1:7]
numricalData_out =  open(VALID_DIR + "/numricalData.pickle","wb")
pickle.dump(numricalData, numricalData_out)
numricalData_out.close()

imgData = []
for index, row in testCSV.iterrows():
  #convert img to data
  mPath = row.image_path.replace("/ATI","")

  # img = image.load_img(mPath, target_size=(HEIGHT, WIDTH))
  # x = image.img_to_array(img)
  # x = np.expand_dims(x, axis=0)
  # x = preprocess_input(x)
  img = cv2.imread(mPath)
  img = cv2.resize(img,(HEIGHT, WIDTH))
  imgData.append(img)

imgData = np.array(imgData)
imgData_out =  open(VALID_DIR + "/imgData.pickle","wb")
pickle.dump(imgData, imgData_out)
imgData_out.close()